In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
#READING IN MAP FILE FOR XIST JUST TO TAKE ITS STRUCTURE, BUT WILL NOT ACTUALLY USE THE DATA



colnames=['nucleotide_number', 'SHAPE_reactivity', 'SE_reactivity', 'nucleotide_identity'] 
df = pd.read_csv("../../../Data/Map files Xist/XIST_exvivo_1M7_rep1.map", sep="\t",names=colnames, header=None,index_col=0)
df.head()

,SHAPE_reactivity,SE_reactivity,nucleotide_identity
nucleotide_number,,,
1,-999.0,0.0,C
2,-999.0,0.0,G
3,-999.0,0.0,G
4,-999.0,0.0,C
5,-999.0,0.0,T


In [3]:
#REMOVE UNNECESSARY COLUMNS
df = df.drop(df[["SHAPE_reactivity","SE_reactivity"]],axis=1)

In [4]:
#Create empty columnS
df["DeltaSHAPE_ex_vivo"] = 0
df["DeltaSHAPE_in_cell"] = 0
#df["DeltaSHAPE"] = 0


In [5]:
df.head()

,nucleotide_identity,DeltaSHAPE_ex_vivo,DeltaSHAPE_in_cell
nucleotide_number,,,
1,C,0,0
2,G,0,0
3,G,0,0
4,C,0,0
5,T,0,0


In [6]:
#READING IN DELTASHAPE INFORMATION, FROM COMPARISONS OF REPLICATES EX VIVO 1 VS IN CELL 1 & EX VIVO 2 VS IN CELL 2  

deltashape = pd.read_csv('../../../Analysis/deltaSHAPE/XIST_exvivo_vs_invivo_reps1both_1M7.txt', sep="\t", header=0, index_col=0)
#invivo = pd.read_csv('../../Data/Map files Xist/XIST_incell_1M7.txt', sep="\t", header=0, index_col=0)


deltashape2 = pd.read_csv('../../../Analysis/deltaSHAPE/XIST_exvivo_vs_invivo_reps2both_IM7.txt', sep="\t", header=0, index_col=0)
#invivo = pd.read_csv('../../Data/Map files Xist/XIST_incell_1M7.txt', sep="\t", header=0, index_col=0)

In [7]:
deltashape2.head()

,Seq,DeltaSHAPE,Z-factor,Std_Score,Smoothed_Data1,Smoothed_Data2,Unsmoothed_Diff,Data1,Data2
Nuc,,,,,,,,,
179,A,-1.525417,0.526480,-1.928230,0.317628,1.843045,-0.808703,0.109179,0.917883
180,A,-1.403042,0.543571,-1.787718,0.199170,1.602212,-3.580023,0.437573,4.017596
181,G,-1.415818,0.525799,-1.802388,0.425973,1.841791,0.179601,0.050757,-0.128844
182,A,-1.232581,0.305541,-1.591995,0.295265,1.527846,-0.847031,0.789590,1.636621
183,T,-1.556549,0.442654,-1.963976,0.292537,1.849086,-3.030313,0.045448,3.075760


In [8]:
#ADD DETALSHAPE INFORMATION TO A NEW DATAFRAME OF RIGHT SIZE i.e. 17918 ROWS x 3 columns AND CALCULATE AVERAGE DELTASHAPE

df_ = pd.DataFrame(index=np.arange(17918), columns=['nucleotide', 'DeltaSHAPE_1','DeltaSHAPE_2'])
df_ = df_.fillna(0) # with 0s rather than NaNs
df_["DeltaSHAPE_1"].loc[deltashape.index] = deltashape["DeltaSHAPE"]
df_["DeltaSHAPE_2"].loc[deltashape2.index] = deltashape2["DeltaSHAPE"]

#ADD AN EXTRA COLUMN FOR CALCULATING AVERAGE OF DELTASHAPE
df_['average_deltaSHAPE'] = 0

#THE IMPORTANT RULE IS: IF ONLY ONE OF THE TWO FILES HAS DELTASHAPE VALUE FOR A GIVEN POSITION, THEN WE 
#DON'T TAKE THE AVERAGE BETWEEN 0 AND THAT VALUE. WE JUST REPORT THE SINGLE VALUE FROM THE ONLY FILE THAT 
#HAS DELTASHAPE INFORMATION FOR THAT NUCLEOTIDE.
for index,row in df_.iterrows():
    if row["DeltaSHAPE_1"] != 0 and row["DeltaSHAPE_2"] == 0:
         df_.loc[row.name,'average_deltaSHAPE'] = row["DeltaSHAPE_1"]
    elif row["DeltaSHAPE_1"] == 0 and row["DeltaSHAPE_2"] != 0:
        df_.loc[row.name,'average_deltaSHAPE'] = row["DeltaSHAPE_2"]
    elif row["DeltaSHAPE_1"] != 0 and row["DeltaSHAPE_2"] != 0:
        a = np.array([row['DeltaSHAPE_1'], row['DeltaSHAPE_2']])
        df_.loc[row.name,'average_deltaSHAPE'] = np.mean(a)



C:\Users\maran\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
#REMOVING UNNECESSARY COLUMNS FROM THE DATAFRAME CONTAINING THE AVERAGE DELTASHAPE
#MERGE THIS DATAFRAME WITH THE PREVIOUS DATAFRAME 

df.reset_index()

new_df = df.join(df_)

new_df = new_df.fillna(0)
new_df = new_df.drop(new_df[["nucleotide","DeltaSHAPE_1","DeltaSHAPE_2"]],axis=1)

In [10]:
new_df.head()

,nucleotide_identity,DeltaSHAPE_ex_vivo,DeltaSHAPE_in_cell,average_deltaSHAPE
nucleotide_number,,,,
1,C,0,0,0.0
2,G,0,0,0.0
3,G,0,0,0.0
4,C,0,0,0.0
5,T,0,0,0.0


In [11]:
# POPULATE COLUMNS DeltaSHAPE_ex_vivo AND DeltaSHAPE_in_cell BASED ON THE VALUE OF THE AVERAGE DELTASHAPE

for index,row in new_df.iterrows():
    if row["average_deltaSHAPE"] > 0:
        new_df.loc[row.name,'DeltaSHAPE_ex_vivo'] = 1
        new_df.loc[row.name,'DeltaSHAPE_in_cell'] = 0
        #print('positive')
    elif row["average_deltaSHAPE"] < 0:
        new_df.loc[row.name,'DeltaSHAPE_ex_vivo'] = 0
        new_df.loc[row.name,'DeltaSHAPE_in_cell'] = 1
        #print('negative')
    else:
        new_df.loc[row.name,'DeltaSHAPE_ex_vivo'] = 0
        new_df.loc[row.name,'DeltaSHAPE_in_cell'] = 0
        #print('neither')

In [12]:
new_df.head()

,nucleotide_identity,DeltaSHAPE_ex_vivo,DeltaSHAPE_in_cell,average_deltaSHAPE
nucleotide_number,,,,
1,C,0,0,0.0
2,G,0,0,0.0
3,G,0,0,0.0
4,C,0,0,0.0
5,T,0,0,0.0


# RELEVANT PART OF JUPYTER NOTEBOOK STARTS HERE 


In [13]:
new_df['deltaSHAPE_confirmed_rep1'] = 0

data = pd.read_csv('../../../deltaSHAPE_confirmed_sites/deltaSHAPE_rep1.igv',sep="\t",index_col=0)
data=data.reset_index()

for j in data.index:
    
    start = int(data.loc[j,"Start"])          
    end = int(data.loc[j,"End"])
    coordinates = np.arange(start,end+1)
    new_df.loc[coordinates,'deltaSHAPE_confirmed_rep1'] = 1
    

print(new_df)


                  nucleotide_identity  DeltaSHAPE_ex_vivo  DeltaSHAPE_in_cell  \
nucleotide_number                                                               
1                                   C                   0                   0   
2                                   G                   0                   0   
3                                   G                   0                   0   
4                                   C                   0                   0   
5                                   T                   0                   0   
6                                   T                   0                   0   
7                                   G                   0                   0   
8                                   C                   0                   0   
9                                   T                   0                   0   
10                                  C                   0                   0   
11                          

In [14]:
new_df['deltaSHAPE_confirmed_rep2'] = 0

data = pd.read_csv('../../../deltaSHAPE_confirmed_sites/deltaSHAPE_rep2.igv',sep="\t",index_col=0)
data=data.reset_index()

for j in data.index:
    
    start = int(data.loc[j,"Start"])          
    end = int(data.loc[j,"End"])
    coordinates = np.arange(start,end+1)
    new_df.loc[coordinates,'deltaSHAPE_confirmed_rep2'] = 1
    

print(new_df)


                  nucleotide_identity  DeltaSHAPE_ex_vivo  DeltaSHAPE_in_cell  \
nucleotide_number                                                               
1                                   C                   0                   0   
2                                   G                   0                   0   
3                                   G                   0                   0   
4                                   C                   0                   0   
5                                   T                   0                   0   
6                                   T                   0                   0   
7                                   G                   0                   0   
8                                   C                   0                   0   
9                                   T                   0                   0   
10                                  C                   0                   0   
11                          

In [15]:
df_1 = pd.read_csv("../../../Xist in vivo vs. ex vivo scaled by dc_diff_BUM_HMM_analysis.txt",sep="\t")
#Select columns
df_1 = df_1[['UM','MU']]

#Rename columns
df_1.columns = ['diffBUMHMM_ex_vivo', 'diffBUMHMM_in_cell']

In [16]:
new_df ["diffBUMHMM_in_cell"] = 0
new_df ['diffBUMHMM_ex_vivo'] = 0

new_df["diffBUMHMM_in_cell"].loc[df_1.index] = df_1["diffBUMHMM_in_cell"]
new_df['diffBUMHMM_ex_vivo'].loc[df_1.index] = df_1['diffBUMHMM_ex_vivo']

In [17]:
print(new_df)

                  nucleotide_identity  DeltaSHAPE_ex_vivo  DeltaSHAPE_in_cell  \
nucleotide_number                                                               
1                                   C                   0                   0   
2                                   G                   0                   0   
3                                   G                   0                   0   
4                                   C                   0                   0   
5                                   T                   0                   0   
6                                   T                   0                   0   
7                                   G                   0                   0   
8                                   C                   0                   0   
9                                   T                   0                   0   
10                                  C                   0                   0   
11                          

In [18]:
exvivodiffbumhmm = new_df[new_df.diffBUMHMM_ex_vivo > 0.9]
incelldiffbumhmm = new_df[new_df.diffBUMHMM_in_cell > 0.9]

In [19]:
print(exvivodiffbumhmm)

                  nucleotide_identity  DeltaSHAPE_ex_vivo  DeltaSHAPE_in_cell  \
nucleotide_number                                                               
63                                  C                   0                   0   
66                                  T                   0                   0   
70                                  A                   0                   0   
72                                  G                   0                   0   
77                                  T                   0                   0   
78                                  T                   0                   0   
79                                  G                   0                   0   
131                                 A                   0                   0   
133                                 C                   0                   0   
134                                 A                   0                   0   
139                         

In [20]:
print(exvivodiffbumhmm.columns)

Index(['nucleotide_identity', 'DeltaSHAPE_ex_vivo', 'DeltaSHAPE_in_cell',
       'average_deltaSHAPE', 'deltaSHAPE_confirmed_rep1',
       'deltaSHAPE_confirmed_rep2', 'diffBUMHMM_in_cell',
       'diffBUMHMM_ex_vivo'],
      dtype='object')


In [21]:
counter = 0
counter2 = 0
for index, row in new_df.iterrows():
    if row['deltaSHAPE_confirmed_rep1'] > 0.0:
        counter = counter+1    
    elif row['deltaSHAPE_confirmed_rep2'] > 0.0:
        counter2= counter2 +1
    else:
        continue
        
print(" total number of dSHAPE confirmed site in rep 1   %s" % counter)
print(" total number of dSHAPE confirmed site in rep 2    %s" % counter2)

 total number of dSHAPE confirmed site in rep 1   1066
 total number of dSHAPE confirmed site in rep 2    989


In [22]:
#Numbrt of deltaSHAPE confirmed sites that correpsond to positions
#with diffBUMHMM posterior probability higher than 0.9 for ex vivo

detected_deltashape_rep1_exvivodiffbumhmm= exvivodiffbumhmm['deltaSHAPE_confirmed_rep1'].sum()
detected_deltashape_rep2_exvivodiffbumhmm= exvivodiffbumhmm['deltaSHAPE_confirmed_rep2'].sum()

#Numbrt of deltaSHAPE confirmed sites that correpsond to positions
#with diffBUMHMM posterior probability higher than 0.9 for incell

detected_deltashape_rep1_incelldiffbumhmm= incelldiffbumhmm['deltaSHAPE_confirmed_rep1'].sum()
detected_deltashape_rep2_incelldiffbumhmm= incelldiffbumhmm['deltaSHAPE_confirmed_rep2'].sum()


In [24]:
#Percentage of confirmed deltaSHAPE value predicted by either diffBUM HMM ev v
#print(exvivodiffbumhmm[exvivodiffbumhmm.deltaSHAPE_confirmed_rep1 >0])

perc_rep1=(detected_deltashape_rep1_exvivodiffbumhmm+detected_deltashape_rep1_incelldiffbumhmm)/1066*100

perc_rep2=(detected_deltashape_rep2_exvivodiffbumhmm+detected_deltashape_rep2_incelldiffbumhmm)/989*100


print("Percentage of agreement deltashape rep 1 with diffbumhmm is " + str(perc_rep1))
print("Percentage of agreement deltashape rep 2 with diffbumhmm is " + str(perc_rep2))

Percentage of agreement deltashape rep 1 with diffbumhmm is 13.03939962476548
Percentage of agreement deltashape rep 2 with diffbumhmm is 17.087967644084934
